In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

In [2]:
# Importamos el módulo con nuestras funciones
import sys
sys.path.append('./../../')

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

#Own Library modules
import src
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.pca import todoJunto
from src.test_algorithms import comparaModelos
from src.test_algorithms.err_relativo import err_relativo

#### Objetivo:

El objetivo de este notebook es comparar los resultados de:
+ PCA de scikit-learn 
+ la implementación de PCA usando la función SVD de numpy.
+ la implementación de PCA usando la implementación del algoritmo Jacobi one-sided para obtener la SVD

In [3]:
df = pd.read_csv('../../data/datos_limpios.csv', encoding = "L1")

# Sección 3. Expectativas educativas

- Tomamos como un primer ejemplo para estas pruebas, la sección 3 de la encuesta.

In [4]:
seccion_3 = df.loc[:, ['p10', 'p11', 'p12', 'p13', 'p14']] ###Se seleccionan las preguntas que conforman esta sección

In [5]:
####Se estandarizan los datos.
scaler = preprocessing.StandardScaler()
seccion_3 = scaler.fit_transform(seccion_3)

## PCA de scikit-learn
Se obtienen los atributos que conforman la función componentes_principales del archivo pca.py

In [6]:
pca, var_exp, comp_prin, val_sing, pca_coef, eigenvalues = todoJunto.PCA_from_sklearn(seccion_3)

In [7]:
pca

PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
    svd_solver='full', tol=0.0, whiten=False)

In [8]:
comp_prin

array([[-2.75115163,  1.03803342],
       [ 1.43930486, -0.27433591],
       [-0.07564678, -1.16356118],
       ...,
       [ 1.37202071, -1.2693296 ],
       [-1.48150682, -0.93830593],
       [-0.54628676, -0.67975472]])

In [9]:
val_sing

array([1180.28600498,  782.39469765])

In [10]:
pca_coef

array([[ 0.11876861,  0.43356271,  0.48797319,  0.53049047,  0.52761674],
       [-0.74579472,  0.45015978,  0.3287432 , -0.199727  , -0.30525963]])

In [11]:
eigenvalues

array([2.48486074, 1.09189113])

In [12]:
var_exp

array([0.49697126, 0.21837784])

## PCA usando SVD de numpy
Ahora se obtienen los PCA a través de la función `svd` del módulo `linalg` de `numpy` .

In [13]:
np_val_sing, np_pca_coef, np_comp_prin, np_var_exp = todoJunto.PCA_from_SVD(seccion_3)

In [14]:
np_val_sing # Valores singulares

array([1180.28600498,  782.39469765])

In [15]:
np_pca_coef # coeficientes

array([[ 0.11876861,  0.43356271,  0.48797319,  0.53049047,  0.52761674],
       [-0.74579472,  0.45015978,  0.3287432 , -0.199727  , -0.30525963]])

In [16]:
np_comp_prin # componentes principales)

array([[-2.75115163,  1.03803342],
       [ 1.43930486, -0.27433591],
       [-0.07564678, -1.16356118],
       ...,
       [ 1.37202071, -1.2693296 ],
       [-1.48150682, -0.93830593],
       [-0.54628676, -0.67975472]])

In [17]:
np_var_exp #Varianza explicada

array([0.49697126, 0.21837784])

Comparemos los resultados obtenidos contra los obtenidos antes mediante `scikit-learn`

In [18]:
#valores singulares
np.allclose(np_val_sing,val_sing)

True

In [19]:
# componentes
np.allclose(np_comp_prin,comp_prin)

True

In [20]:
# Varianza explicada
np.allclose(np_var_exp,var_exp)

True

Vemos que, luego de la última actualización a la función `PCA_from_SVD`, los coeficientes ya resultan iguales

In [21]:
# coeficientes
np.allclose(np_pca_coef,pca_coef)

True

In [22]:
pca_coef

array([[ 0.11876861,  0.43356271,  0.48797319,  0.53049047,  0.52761674],
       [-0.74579472,  0.45015978,  0.3287432 , -0.199727  , -0.30525963]])

In [23]:
np_pca_coef

array([[ 0.11876861,  0.43356271,  0.48797319,  0.53049047,  0.52761674],
       [-0.74579472,  0.45015978,  0.3287432 , -0.199727  , -0.30525963]])

Podemos ver un resumen de lo anterior mediante la función `compara_resultados_sk_np`

In [24]:
comparaModelos.compara_resultados_sk_np(seccion_3)

,elemento,Igualdad,Max error relativo,Error relativo
0,varianza explicada,True,1.270989e-16,"[0.0, 1.2709886748169263e-16]"
1,valores singulares,True,0.000000e+00,"[0.0, 0.0]"
2,coeficientes,True,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0...."
3,componentes principales,True,1.254858e-09,"[[7.996555014011452e-12, 1.1263374054990842e-1..."


## PCA usando SVD con aproximación por Jacobi one-sided
Ahora se obtienen los PCA a través de la aproximación a la  `svd` mediante el algoritmo Jacobi one-sided.

In [25]:
jac_val_sing, jac_pca_coef, jac_comp_prin, jac_var_exp = todoJunto.PCA_from_SVD_jacobi(seccion_3)

Comparemos los resultados obtenidos 

In [26]:
#valores singulares
np.allclose(jac_val_sing,val_sing)

False

In [27]:
jac_val_sing

array([1158.20422871,  742.90658598])

In [28]:
val_sing

array([1180.28600498,  782.39469765])

Veamos el error relativo en los valores singulares:

In [29]:
err = err_relativo(jac_val_sing, val_sing) 
err

array([0.01870884, 0.05047083])

Los errores resultan del orden de $10^{-2}$.

Tampoco los coeficientes son iguales, veaos qué tan cercanos resultan:

In [30]:
# coeficientes
np.allclose(jac_pca_coef,pca_coef)

False

In [31]:
jac_pca_coef

array([[ 0.2298819 ,  0.42540564, -0.85353058, -0.17182733, -0.09025116],
       [ 0.39996804, -0.24768259, -0.1854867 ,  0.86197807, -0.03560042]])

In [32]:
pca_coef

array([[ 0.11876861,  0.43356271,  0.48797319,  0.53049047,  0.52761674],
       [-0.74579472,  0.45015978,  0.3287432 , -0.199727  , -0.30525963]])

In [33]:
err_coef = err_relativo(jac_pca_coef, pca_coef) 
err_coef

array([[0.93554427, 0.01881405, 2.74913417, 1.32390276, 1.17105439],
       [1.53629776, 1.55021041, 1.56422976, 5.31578144, 0.8833766 ]])

Los errores relativos ya rondan el 1, llegando hasta 5.31.

Comparemos ahora las componentes resultantes con las obtenidas mediante sklearn, así como sus errores relativos:

In [34]:
# componentes
np.allclose(jac_comp_prin,comp_prin)

False

In [35]:
jac_comp_prin

array([[ 0.00240224,  0.00030158],
       [-0.00055665, -0.00158775],
       [-0.00019087,  0.00015338],
       ...,
       [-0.00105638,  0.00021091],
       [ 0.00133393, -0.00038486],
       [ 0.00029679,  0.00134427]])

In [36]:
comp_prin

array([[-2.75115163,  1.03803342],
       [ 1.43930486, -0.27433591],
       [-0.07564678, -1.16356118],
       ...,
       [ 1.37202071, -1.2693296 ],
       [-1.48150682, -0.93830593],
       [-0.54628676, -0.67975472]])

In [37]:
err_comp = err_relativo(jac_comp_prin, comp_prin) 
np.amax(err_comp)

3.3713442217953022

El mayor error es considerablemente elevado, aproximadamente 3.37.

Por último, repetimos el análisis para la varianza explicada:

In [38]:
# Varianza explicada
np.allclose(jac_var_exp,var_exp)

False

In [39]:
jac_var_exp

array([0.47854971, 0.19689069])

In [40]:
var_exp

array([0.49697126, 0.21837784])

In [41]:
err_comp = err_relativo(jac_var_exp, var_exp)
err_comp

array([0.03706765, 0.09839436])

Una vez más, los errores relativos son del orden de $10^{-2}$.

Esta vez, observamos que todos los elementos resultaron distintos, veamos el resumen de los errores relativos gracias a la función `compara_resultados_sk_jac`

In [42]:
comparaModelos.compara_resultados_sk_jac(seccion_3)

,elemento,Igualdad,Max error relativo,Error relativo
0,varianza explicada,False,1.091324e-01,"[0.038494552337125544, 0.10913236956981359]"
1,valores singulares,False,5.315354e-02,"[0.01906552897109906, 0.05315353561094553]"
2,coeficientes,False,7.574608e+00,"[[0.4833494555008358, 0.019174802487839707, 1...."
3,componentes principales,False,2.218037e+08,"[[1146.2424262714808, 3440.958961197482], [258..."


Aunque para varianza explpicada y valores singulares los errores son del orden de $10^{-2}$, ya a nivel de coeficientes y componentes principales se ven magnificados, llegando al orden de $10^8$ que es demasiado grande.

Observamos que la matriz utilizada es medianamente bien condicionada, lo que puede explicar la magnitud de los errores obtenidos:

In [43]:
np.linalg.cond(seccion_3)

3.5192351424081814

A continuación, realizamos una comparación similar para las demás secciones, apoyándonos de las funciones `compara_resultados_sk_np` y `compara_resultados_sk_jac`. Como vimos antes, la primera compara los resultados obtenidos a través de scikit learn con los obtenidos al aplicar PCA usando la SVD de numpy; mientras tanto, la segunda compara los resultados de scikit learn con los obtenidos al aplicar PCA usando la SVD aproximada a través del algoritmo Jacobi one-sided.

## PCA usando algoritmo QR
Ahora se obtienen los PCA a través de la función implementada `PCA_from_QR`.

In [44]:
qr_val_sing, qr_pca_coef, qr_comp_prin, qr_var_exp = todoJunto.PCA_from_QR_vf(seccion_3)

se alcanzo la tolerancia deseada


In [45]:
qr_val_sing # Valores singulares

array([2.48486074, 1.09189113])

In [46]:
qr_pca_coef # coeficientes

array([[ 0.11876861,  0.43356271,  0.48797319,  0.53049047,  0.52761674],
       [-0.74579473,  0.45015977,  0.3287432 , -0.19972699, -0.30525963]])

In [47]:
qr_comp_prin # componentes principales)

array([[-2.75115163,  1.03803342],
       [ 1.43930486, -0.27433593],
       [-0.07564678, -1.16356118],
       ...,
       [ 1.37202071, -1.2693296 ],
       [-1.48150682, -0.93830594],
       [-0.54628676, -0.67975471]])

In [48]:
qr_var_exp #Varianza explicada

array([0.49697126, 0.21837784])

Comparemos los resultados obtenidos contra los obtenidos antes mediante `scikit-learn`

In [49]:
#valores singulares
qr_val_sing
val_sing

array([1180.28600498,  782.39469765])

In [50]:
# componentes
np.allclose(qr_comp_prin,comp_prin)

True

In [51]:
# Varianza explicada
np.allclose(qr_var_exp,var_exp)

True

In [52]:
# coeficientes
np.allclose(qr_pca_coef,pca_coef)

True

In [53]:
pca_coef

array([[ 0.11876861,  0.43356271,  0.48797319,  0.53049047,  0.52761674],
       [-0.74579472,  0.45015978,  0.3287432 , -0.199727  , -0.30525963]])

In [54]:
qr_pca_coef

array([[ 0.11876861,  0.43356271,  0.48797319,  0.53049047,  0.52761674],
       [-0.74579473,  0.45015977,  0.3287432 , -0.19972699, -0.30525963]])

Podemos ver un resumen de lo anterior mediante la función `compara_resultados_sk_np`

In [180]:
df_seccion_3_qr = comparaModelos.compara_resultados_abs_sk_qr(seccion_3)
df_seccion_3_qr.insert(0, "Seccion", np.repeat("seccion_3",4), True) 
df_seccion_3_qr

se alcanzo la tolerancia deseada


,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_3,varianza explicada,True,3.164762e-14,"[2.5690750673682666e-15, 3.164761800294246e-14]"
1,seccion_3,eigenvalores,True,5.348311e-14,"[1.9301538251445997e-14, 5.3483108037220185e-14]"
2,seccion_3,coeficientes,True,3.237183e-08,"[[1.5891228936468213e-13, 1.5184937284479278e-..."
3,seccion_3,componentes principales,True,1.026288e-05,"[[7.979283139599526e-12, 2.1940877302197085e-0..."


In [169]:
np.linalg.cond(seccion_1)

11.375002491295175

# Sección 1. Datos generales

In [56]:
seccion_1 = df.loc[:, ['p1', 'p2', 'p3_A', 'p3_B','p3_C','p4_A', 'p4_B','p4_C','p5_A', 'p5_B','p5_C']] ###Se seleccionan las preguntas que conforman esta sección

In [57]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()

seccion_1 = pd.DataFrame(scaler.fit_transform(seccion_1))

### Scikit learn vs PCA a partir de SVD de numpy

In [58]:
df_seccion_1 = comparaModelos.compara_resultados_abs_sk_np(seccion_1)
df_seccion_1.insert(0, "Seccion",np.repeat("seccion_1",4), True) 
df_seccion_1

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_1,varianza explicada,True,1.845437e-16,"[1.8454368235801489e-16, 1.5513265898145355e-1..."
1,seccion_1,valores singulares,True,0.000000e+00,"[0.0, 0.0, 0.0]"
2,seccion_1,coeficientes,True,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,seccion_1,componentes principales,True,1.328783e-09,"[[2.6344109242589875e-11, 2.6483430011396585e-..."


### Scikit learn vs PCA a partir de SVD aproximada mediante Jacobi-one sided

In [59]:
df_seccion_1_jac = comparaModelos.compara_resultados_abs_sk_jac(seccion_1)
df_seccion_1_jac.insert(0, "Seccion", np.repeat("seccion_1",4), True) 
df_seccion_1_jac

Nota: Los resultados son de distinta longitud y por lo tanto se comparan solo las entradas en común


,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_1,varianza explicada,False,5.597746e-01,"[0.5597745591370169, 0.028790013244885136, 0.0..."
1,seccion_1,valores singulares,False,2.489093e-01,"[0.24890934784595498, 0.014500133559060375, 0...."
2,seccion_1,coeficientes,False,7.746779e+02,"[[0.8899611016386196, 0.7850258673183417, 0.79..."
3,seccion_1,componentes principales,False,6.549034e+06,"[[739.0213528847813, 30.11391147729844, 1154.1..."


### Scikit learn vs PCA a partir del algoritmo QR

In [161]:
df_seccion_1_qr = comparaModelos.compara_resultados_abs_sk_qr(seccion_1)
df_seccion_1_qr.insert(0, "Seccion", np.repeat("seccion_1",4), True) 
df_seccion_1_qr

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_1,varianza explicada,True,1.357242e-13,"[6.016124044870922e-14, 2.869954191156973e-14,..."
1,seccion_1,eigenvalores,True,1.402497e-13,"[6.469083362511687e-14, 2.414423985846565e-14,..."
2,seccion_1,coeficientes,True,8.808192e-06,"[[1.716471218079173e-14, 1.8088529315322355e-1..."
3,seccion_1,componentes principales,False,1.790270e-04,"[[2.6356809626567568e-11, 6.134861477385035e-0..."


In [167]:
np.linalg.cond(seccion_1)

11.375002491295175

# Sección 2. Situación laboral 

In [64]:
seccion_2 = df.loc[:, ['p6', 'p7', 'p8', 'p9_A', 'p9_B', 'p9_C', 'p9_D', 'p9_E', 'p9_F', 'p9_G']] ###Se seleccionan las preguntas que conforman esta sección

In [65]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()

seccion_2 = pd.DataFrame(scaler.fit_transform(seccion_2))

### Scikit learn vs PCA a partir de SVD de numpy

In [66]:
df_seccion_2 = comparaModelos.compara_resultados_abs_sk_np(seccion_2)
df_seccion_2.insert(0, "Seccion", np.repeat("seccion_2",4), True) 
df_seccion_2

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_2,varianza explicada,True,0.000000e+00,"[0.0, 0.0, 0.0]"
1,seccion_2,valores singulares,True,0.000000e+00,"[0.0, 0.0, 0.0]"
2,seccion_2,coeficientes,True,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,seccion_2,componentes principales,True,1.725904e-07,"[[6.02594840918337e-11, 2.0427044636858444e-09..."


### Scikit learn vs PCA a partir de SVD aproximada mediante Jacobi-one sided

In [67]:
df_seccion_2_jac = comparaModelos.compara_resultados_abs_sk_jac(seccion_2)
df_seccion_2_jac.insert(0, "Seccion", np.repeat("seccion_2",4), True) 
df_seccion_2_jac

Nota: Los resultados son de distinta longitud y por lo tanto se comparan solo las entradas en común


,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_2,varianza explicada,False,2.048381e-01,"[0.126307315429458, 0.20483809151015386, 0.114..."
1,seccion_2,valores singulares,False,1.082815e-01,"[0.06127626725063698, 0.10828148584329743, 0.0..."
2,seccion_2,coeficientes,False,3.730775e+01,"[[0.15104388965092766, 0.9311740542814365, 37...."
3,seccion_2,componentes principales,False,2.998254e+06,"[[3380.405114797662, 55.090909932104154, 0.020..."


### Scikit learn vs PCA a partir del algoritmo QR

In [163]:
df_seccion_2_qr = comparaModelos.compara_resultados_abs_sk_qr(seccion_2)
df_seccion_2_qr.insert(0, "Seccion", np.repeat("seccion_2",4), True) 
df_seccion_2_qr

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_2,varianza explicada,True,0.000006,"[1.426533869698987e-13, 6.0164012023457805e-06..."
1,seccion_2,eigenvalores,True,0.000006,"[1.3636577603894522e-13, 6.016401196106755e-06..."
2,seccion_2,coeficientes,False,0.213514,"[[9.63301597215641e-14, 2.389524864392304e-13,..."
3,seccion_2,componentes principales,False,2.513791,"[[6.024223679040742e-11, 0.0001648771947033299..."


In [170]:
np.linalg.cond(seccion_2)

23.81611840914737

# Sección 4. Características personales

In [69]:
seccion_4 = df.loc[:, ['p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27',
                         'p28', 'p29', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p40',
                         'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p50','p51', 'p52', 'p53', 
                         'p54', 'p55', 'p56']] ###Se seleccionan las preguntas que conforman esta sección

In [70]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()

seccion_4 = pd.DataFrame(scaler.fit_transform(seccion_4))

### Scikit learn vs PCA a partir de SVD de numpy

In [71]:
comparaModelos.compara_resultados_sk_np(seccion_4)

,elemento,Igualdad,Max error relativo,Error relativo
0,varianza explicada,True,3.810244e-16,"[1.8163619603877741e-16, 2.6276477052749485e-1..."
1,valores singulares,True,0.000000e+00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,coeficientes,False,2.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,componentes principales,False,2.000000e+00,"[[7.267035848063278e-11, 5.408932705117198e-12..."


Revisemos las diferencias obtenidas:

In [72]:
# PCA de scikit learn
pca, var_exp, comp_prin, val_sing, pca_coef, eigenvalues = todoJunto.PCA_from_sklearn(seccion_4)

# PCA a partir de SVD de numpy
np_val_sing, np_pca_coef, np_comp_prin, np_var_exp = todoJunto.PCA_from_SVD(seccion_4)

Comparamos primero los coeficientes:

In [73]:
pca_coef == np_pca_coef

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True,  True,  True,  True,  True],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, Fal

Lo anterior indica que las diferencias ocurren únicamente en algunas de las entradas, y podemos comprobar que se trata sólo de diferencias de signo:

In [74]:
pca_coef[2]

array([-0.02794204,  0.05525299,  0.02325172,  0.03993592,  0.0338589 ,
        0.02343367,  0.07592834,  0.06985281, -0.05350466, -0.11115483,
       -0.10704293, -0.07664377, -0.16954962, -0.04327191, -0.05971324,
       -0.01991098, -0.16330696, -0.05474756, -0.14034013, -0.28164905,
       -0.23748561, -0.22211957, -0.24550437, -0.22538582, -0.1962033 ,
       -0.11009484,  0.16075068,  0.17153375,  0.16529315,  0.14000056,
        0.18666929,  0.28301588,  0.22302426,  0.31955116,  0.22370007,
        0.19366366,  0.1529959 ,  0.08369772,  0.09539056,  0.1047206 ,
        0.08806734,  0.06067474])

In [75]:
np_pca_coef[2]

array([ 0.02794204, -0.05525299, -0.02325172, -0.03993592, -0.0338589 ,
       -0.02343367, -0.07592834, -0.06985281,  0.05350466,  0.11115483,
        0.10704293,  0.07664377,  0.16954962,  0.04327191,  0.05971324,
        0.01991098,  0.16330696,  0.05474756,  0.14034013,  0.28164905,
        0.23748561,  0.22211957,  0.24550437,  0.22538582,  0.1962033 ,
        0.11009484, -0.16075068, -0.17153375, -0.16529315, -0.14000056,
       -0.18666929, -0.28301588, -0.22302426, -0.31955116, -0.22370007,
       -0.19366366, -0.1529959 , -0.08369772, -0.09539056, -0.1047206 ,
       -0.08806734, -0.06067474])

In [76]:
pca_coef[2] == -np_pca_coef[2]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

Lo anterior nos indica que podríamos obtener también diferencias de signo en las componentes correspondientes. En este caso, nos apoyamos en la función `compara_resultados_abs_sk_np` que compara los valores absolutos de los coeficientes y componentes que previamente nos resultaron distintos y vemos que esta vez obtenemos la igualdad:

In [77]:
df_seccion_4 = comparaModelos.compara_resultados_abs_sk_np(seccion_4)
df_seccion_4.insert(0, "Seccion", np.repeat("seccion_4",4), True) 
df_seccion_4

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_4,varianza explicada,True,3.810244e-16,"[1.8163619603877741e-16, 2.6276477052749485e-1..."
1,seccion_4,valores singulares,True,0.000000e+00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,seccion_4,coeficientes,True,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,seccion_4,componentes principales,True,7.415905e-08,"[[7.267035848063278e-11, 5.408932705117198e-12..."


En adelante, usaremos esta función de comparación, para no tomar en cuenta las diferencias de signos.

### Scikit learn vs PCA a partir de SVD aproximada mediante Jacobi-one sided

En vista de lo observado en la sección anterior, a partir de ahora usaremos la función `compara_resultados_abs_sk_jac` que no toma en cuenta los signos:

In [78]:
df_seccion_4_jac = comparaModelos.compara_resultados_abs_sk_jac(seccion_4)
df_seccion_4_jac.insert(0, "Seccion",np.repeat("seccion_4",4), True) 
df_seccion_4_jac

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_4,varianza explicada,False,5.161847e-02,"[0.04112376947389873, 0.03252009150507147, 0.0..."
1,seccion_4,valores singulares,False,2.548450e-02,"[0.02035472727571438, 0.01639443449372074, 0.0..."
2,seccion_4,coeficientes,False,6.948794e+02,"[[0.018741068235355388, 0.035586748186205185, ..."
3,seccion_4,componentes principales,False,2.062674e+10,"[[1067.708263034393, 4624.663310507209, 1086.4..."


De nuevo, obtenemos errores del orden de $10^{-2}$ en varianza explicada y valores singulares, que se transforman en errores mayores a 1 en coeficientes y componentes principales. Además, comprobamos que seguimos trabajando con una matriz medianamente bien condicionada:

In [79]:
np.linalg.cond(seccion_4)

3.9339555866144793

### Scikit learn vs PCA a partir del algoritmo QR

In [80]:
df_seccion_4_qr = comparaModelos.compara_resultados_abs_sk_qr(seccion_4)
df_seccion_4_qr.insert(0, "Seccion", np.repeat("seccion_4",4), True) 
df_seccion_4_qr

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_4,varianza explicada,False,0.003287,"[9.081809801938787e-15, 2.7590300905387026e-15..."
1,seccion_4,eigenvalores,False,0.003287,"[6.919461792403764e-15, 5.00503432054534e-15, ..."
2,seccion_4,coeficientes,False,55.381394,"[[5.809725747878342e-15, 6.155074707948904e-14..."
3,seccion_4,componentes principales,False,362438.355853,"[[7.27210338561804e-11, 5.398044704011588e-12,..."


In [171]:
np.linalg.cond(seccion_4)

3.9339555866144793

# Sección 5. Inglés y cómputo

## 5.1. Califica tu nivel de habilidad para leer en inglés.

In [81]:
np.random.seed(150) ###Se define la semilla para la reproducibilidad de los resultados

seccion_5_1 = df.loc[:, ['p57', 'p58', 'p59', 'p60']] ###Se seleccionan las preguntas que conforman esta sección

In [82]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()

seccion_5_1 = pd.DataFrame(scaler.fit_transform(seccion_5_1))

### Scikit learn vs PCA a partir de SVD de numpy

In [83]:
df_seccion_5_1 = comparaModelos.compara_resultados_abs_sk_np(seccion_5_1)
df_seccion_5_1.insert(0, "Seccion", np.repeat("seccion_5_1",4), True) 
df_seccion_5_1

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_5_1,varianza explicada,True,2.971219e-16,[2.9712188186743343e-16]
1,seccion_5_1,valores singulares,True,0.000000e+00,[0.0]
2,seccion_5_1,coeficientes,True,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0]]"
3,seccion_5_1,componentes principales,True,1.369991e-10,"[[1.0864534238555451e-10], [1.3710478122857844..."


### Scikit learn vs PCA a partir de SVD aproximada mediante Jacobi-one sided

In [84]:
df_seccion_5_1_jac = comparaModelos.compara_resultados_abs_sk_jac(seccion_5_1)
df_seccion_5_1_jac.insert(0, "Seccion", np.repeat("seccion_5_1",4), True) 
df_seccion_5_1_jac

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_5_1,varianza explicada,False,0.000068,[6.779697210380624e-05]
1,seccion_5_1,valores singulares,False,0.000034,[3.389791155141556e-05]
2,seccion_5_1,coeficientes,False,6.011951,"[[0.013487521683155597, 0.07487148385623248, 6..."
3,seccion_5_1,componentes principales,False,106185.377922,"[[2176.547143604811], [1185.3528509361165], [1..."


### Scikit learn vs PCA a partir del algoritmo QR

In [85]:
df_seccion_5_1_qr = comparaModelos.compara_resultados_abs_sk_qr(seccion_5_1)
df_seccion_5_1_qr.insert(0, "Seccion", np.repeat("seccion_5_1",4), True) 
df_seccion_5_1_qr

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_5_1,varianza explicada,True,7.026933e-14,[7.026932506165296e-14]
1,seccion_5_1,eigenvalores,True,1.384585e-13,[1.3845854997846134e-13]
2,seccion_5_1,coeficientes,True,2.924084e-13,"[[2.9240838097875275e-13, 3.002608367843295e-1..."
3,seccion_5_1,componentes principales,True,1.086433e-10,"[[1.086432785018871e-10], [1.3621290702737174e..."


## 5.2. Con qué frecuencia realizas las siguientes actividades en la computadora

In [86]:
np.random.seed(150) ###Se define la semilla para la reproducibilidad de los resultados

seccion_5_2 = df.loc[:, ['p61', 'p62', 'p63', 'p64']] ###Se seleccionan las preguntas que conforman esta sección

In [87]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()

seccion_5_2 = pd.DataFrame(scaler.fit_transform(seccion_5_2))

In [88]:
df_seccion_5_2 = comparaModelos.compara_resultados_abs_sk_np(seccion_5_2)
df_seccion_5_2.insert(0, "Seccion", np.repeat("seccion_5_2",4), True) 
df_seccion_5_2

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_5_2,varianza explicada,True,1.873993e-16,"[1.873992524757673e-16, 0.0]"
1,seccion_5_2,valores singulares,True,0.000000e+00,"[0.0, 0.0]"
2,seccion_5_2,coeficientes,True,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]"
3,seccion_5_2,componentes principales,True,1.499040e-09,"[[6.788500778143712e-11, 5.432152614613707e-12..."


### Scikit learn vs PCA a partir de SVD aproximada mediante Jacobi-one sided

In [89]:
df_seccion_5_2_jac = comparaModelos.compara_resultados_abs_sk_jac(seccion_5_2)
df_seccion_5_2_jac.insert(0, "Seccion", np.repeat("seccion_5_2",4), True) 
df_seccion_5_2_jac

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_5_2,varianza explicada,False,0.004216,"[0.00421576487797782, 0.003283532492182103]"
1,seccion_5_2,valores singulares,False,0.002106,"[0.0021056655253422533, 0.001643116161445561]"
2,seccion_5_2,coeficientes,False,2.427195,"[[0.12214631300002518, 0.2137900143185933, 0.1..."
3,seccion_5_2,componentes principales,False,337488.585576,"[[3516.789333221952, 757.0460589487742], [3877..."


### Scikit learn vs PCA a partir del algoritmo QR

In [90]:
df_seccion_5_2_qr = comparaModelos.compara_resultados_abs_sk_qr(seccion_5_2)
df_seccion_5_2_qr.insert(0, "Seccion", np.repeat("seccion_5_2",4), True) 
df_seccion_5_2_qr

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_5_2,varianza explicada,True,7.398868e-14,"[8.058167856458061e-15, 7.398868441239215e-14]"
1,seccion_5_2,eigenvalores,True,6.158501e-14,"[2.0426482084662247e-14, 6.158501343219229e-14]"
2,seccion_5_2,coeficientes,True,2.069570e-12,"[[5.488142467074827e-14, 9.080326838272184e-14..."
3,seccion_5_2,componentes principales,True,9.057784e-10,"[[6.780675230636451e-11, 5.478113032138733e-12..."


# Sección 6. Actividades deportivas y culturales

In [91]:
seccion_6 = df.loc[:, ['p65', 'p66']] ###Se seleccionan las preguntas que conforman esta sección

In [92]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()

seccion_6 = pd.DataFrame(scaler.fit_transform(seccion_6))

### Scikit learn vs PCA a partir de SVD de numpy

In [93]:
df_seccion_6 = comparaModelos.compara_resultados_abs_sk_np(seccion_6)
df_seccion_6.insert(0, "Seccion", np.repeat("seccion_6",4), True) 
df_seccion_6

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_6,varianza explicada,True,1.311963e-16,"[0.0, 1.3119626812763805e-16]"
1,seccion_6,valores singulares,True,0.000000e+00,"[0.0, 0.0]"
2,seccion_6,coeficientes,True,0.000000e+00,"[[0.0, 0.0], [0.0, 0.0]]"
3,seccion_6,componentes principales,True,1.080050e-12,"[[5.192048991001361e-13, 1.0800503623567881e-1..."


### Scikit learn vs PCA a partir de SVD aproximada mediante Jacobi-one sided

In [94]:
df_seccion_6_jac = comparaModelos.compara_resultados_abs_sk_jac(seccion_6)
df_seccion_6_jac.insert(0, "Seccion", np.repeat("seccion_6",4), True) 
df_seccion_6_jac

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_6,varianza explicada,True,1.705551e-15,"[1.1547088008302923e-15, 1.7055514856592915e-15]"
1,seccion_6,valores singulares,True,4.947458e-15,"[4.947457968221809e-15, 3.466159070087135e-15]"
2,seccion_6,coeficientes,True,8.635509e-15,"[[8.635508522760138e-15, 8.478499276892998e-15..."
3,seccion_6,componentes principales,False,4.853796e+04,"[[761.0656750587331, 944.3492894702955], [86.4..."


In [95]:
np.linalg.cond(seccion_6)

1.1676565663790537

### Scikit learn vs PCA a partir del algoritmo QR

In [96]:
df_seccion_6_qr = comparaModelos.compara_resultados_abs_sk_qr(seccion_6)
df_seccion_6_qr.insert(0, "Seccion", np.repeat("seccion_6",4), True) 
df_seccion_6_qr

se alcanzo la tolerancia deseada


,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_6,varianza explicada,True,1.574355e-15,"[1.1547088008302923e-15, 1.574355217531654e-15]"
1,seccion_6,eigenvalores,True,9.815007e-15,"[9.815007299798092e-15, 7.08458584195003e-15]"
2,seccion_6,coeficientes,True,1.139759e-08,"[[1.1397585057405627e-08, 1.1397583747124481e-..."
3,seccion_6,componentes principales,True,1.016404e-06,"[[4.807996512030944e-09, 2.7022518785506908e-0..."


# Sección 7. Trayectoria académica

## 7.1. Primaria y secundaria

In [97]:
seccion_7_1 = df.loc[:, ['p67_A','p67_B', 'p67_C', 'p67_D', 'p67_E', 'p68', 'p69', 'p70_A', 'p70_B',
                         'p70_C', 'p70_D', 'p70_E', 'p70_F', 'p70_G','p70_H', 'p71']] ###Se seleccionan las preguntas que conforman esta sección

In [98]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()

seccion_7_1 = pd.DataFrame(scaler.fit_transform(seccion_7_1))

### Scikit learn vs PCA a partir de SVD de numpy

In [99]:
df_seccion_7_1 = comparaModelos.compara_resultados_abs_sk_np(seccion_7_1)
df_seccion_7_1.insert(0, "Seccion", np.repeat("seccion_7_1",4), True) 
df_seccion_7_1

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_7_1,varianza explicada,True,2.201664e-16,"[0.0, 1.1251842512572906e-16, 1.42763830014617..."
1,seccion_7_1,valores singulares,True,0.000000e+00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,seccion_7_1,coeficientes,True,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,seccion_7_1,componentes principales,True,1.700735e-09,"[[8.415655220333915e-11, 6.328821898554283e-11..."


### Scikit learn vs PCA a partir de SVD aproximada mediante Jacobi-one sided

In [100]:
df_seccion_7_1_jac = comparaModelos.compara_resultados_abs_sk_jac(seccion_7_1)
df_seccion_7_1_jac.insert(0, "Seccion", np.repeat("seccion_7_1",4), True) 
df_seccion_7_1_jac

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_7_1,varianza explicada,False,2.170579e-01,"[0.21705791325173573, 0.07273377742978161, 0.1..."
1,seccion_7_1,valores singulares,False,1.032035e-01,"[0.10320347771919386, 0.035728621517128425, 0...."
2,seccion_7_1,coeficientes,False,6.989034e+01,"[[0.05572941838065679, 0.23890575665922156, 0...."
3,seccion_7_1,componentes principales,False,6.646681e+06,"[[5278.56772669397, 2499.8443701227034, 29652...."


### Scikit learn vs PCA a partir del algoritmo QR

In [101]:
df_seccion_7_1_qr = comparaModelos.compara_resultados_abs_sk_qr(seccion_7_1)
df_seccion_7_1_qr.insert(0, "Seccion", np.repeat("seccion_7_1",4), True) 
df_seccion_7_1_qr

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_7_1,varianza explicada,False,0.001526,"[6.593987576421898e-14, 5.513402831160754e-15,..."
1,seccion_7_1,eigenvalores,False,0.001526,"[6.870259717438163e-14, 2.5879191617594965e-15..."
2,seccion_7_1,coeficientes,False,92.114445,"[[4.938881230546565e-10, 1.1860579865672855e-1..."
3,seccion_7_1,componentes principales,False,597.978641,"[[2.761215552029329e-10, 1.3308003747017288e-1..."


In [172]:
np.linalg.cond(seccion_7_1)

13.978669148832259

## 7.2. Educación media superior (Bachillerato)

In [102]:
np.random.seed(150) ###Se define la semilla para la reproducibilidad de los resultados

seccion_7_2 = df.loc[:, ['p72', 'p73', 'p74_A','p74_B','p74_C','p74_D', 'p74_E', 'p74_F', 'p75', 'p76', 'p77', 'p78']] ###Se seleccionan las preguntas que conforman esta sección

In [103]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()

seccion_7_2 = pd.DataFrame(scaler.fit_transform(seccion_7_2))

### Scikit learn vs PCA a partir de SVD de numpy

In [104]:
df_seccion_7_2 = comparaModelos.compara_resultados_abs_sk_np(seccion_7_2)
df_seccion_7_2.insert(0, "Seccion", np.repeat("seccion_7_2",4), True) 
df_seccion_7_2

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_7_2,varianza explicada,True,2.178309e-16,"[2.178308965095565e-16, 1.2571561853136868e-16..."
1,seccion_7_2,valores singulares,True,0.000000e+00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
2,seccion_7_2,coeficientes,True,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,seccion_7_2,componentes principales,True,7.323949e-10,"[[9.761226003432158e-14, 1.0101021943151365e-1..."


### Scikit learn vs PCA a partir de SVD aproximada mediante Jacobi-one sided

In [105]:
df_seccion_7_2_jac = comparaModelos.compara_resultados_abs_sk_jac(seccion_7_2)
df_seccion_7_2_jac.insert(0, "Seccion", np.repeat("seccion_7_2",4), True) 
df_seccion_7_2_jac

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_7_2,varianza explicada,False,9.193659e-02,"[0.07403009976843317, 0.03310760440223537, 0.0..."
1,seccion_7_2,valores singulares,False,4.495770e-02,"[0.03635423469409431, 0.01669313253810829, 0.0..."
2,seccion_7_2,coeficientes,False,2.987641e+01,"[[0.6674372806598604, 0.4995088801926291, 1.07..."
3,seccion_7_2,componentes principales,False,4.885065e+07,"[[1092.6131895026354, 1179.0850653572131, 1280..."


### Scikit learn vs PCA a partir del algoritmo QR

In [173]:
df_seccion_7_2_qr = comparaModelos.compara_resultados_abs_sk_qr(seccion_7_2)
df_seccion_7_2_qr.insert(0, "Seccion", np.repeat("seccion_7_2",4), True) 
df_seccion_7_2_qr

C:\Users\elyes\Anaconda3\envs\mno2020\lib\site-packages\numpy\core\numeric.py:906: RuntimeWarning: overflow encountered in multiply
  return multiply(a.ravel()[:, newaxis], b.ravel()[newaxis, :], out)
./../..\src\pca\funciones_examen_QR.py:143: RuntimeWarning: invalid value encountered in multiply
  Arv[j:m,j:n]=Arv[j:m,j:n]-beta*np.outer(v,(np.transpose(v)@Arv[j:m,j:n]))
./../..\src\pca\funciones_examen_QR.py:234: RuntimeWarning: invalid value encountered in multiply
  Qj[(j-1):m,(j-1):m]=np.eye(m-(j-1))-beta*np.outer(v,v)
./../..\src\pca\todoJunto.py:188: RuntimeWarning: invalid value encountered in less
  conteo = (varianza_acumulada)  <  0.6


,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_7_2,varianza explicada,False,NaN,"[nan, nan, nan, nan, nan, nan]"
1,seccion_7_2,eigenvalores,False,NaN,"[nan, nan, nan, nan, nan, nan]"
2,seccion_7_2,coeficientes,False,NaN,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
3,seccion_7_2,componentes principales,False,NaN,"[[nan, nan, nan, nan, nan, nan], [nan, nan, na..."


In [174]:
np.linalg.cond(seccion_7_2)

7.937439147230621

# Sección 8. Entorno escolar 

## 8.1. Entorno escolar

In [107]:
np.random.seed(150) ###Se define la semilla para la reproducibilidad de los resultados

seccion_8_1 = df.loc[:, ['p79', 'p80', 'p81', 'p82', 'p83', 'p84']] ###Se seleccionan las preguntas que conforman esta sección

In [108]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()

seccion_8_1 = pd.DataFrame(scaler.fit_transform(seccion_8_1))

### Scikit learn vs PCA a partir de SVD de numpy

In [109]:
df_seccion_8_1 = comparaModelos.compara_resultados_abs_sk_np(seccion_8_1)
df_seccion_8_1.insert(0, "Seccion", np.repeat("seccion_8_1",4), True) 
df_seccion_8_1

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_8_1,varianza explicada,True,3.055070e-16,"[2.444284208275238e-16, 3.055070401062262e-16,..."
1,seccion_8_1,valores singulares,True,0.000000e+00,"[0.0, 0.0, 0.0, 0.0]"
2,seccion_8_1,coeficientes,True,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0...."
3,seccion_8_1,componentes principales,True,7.033656e-11,"[[2.8732286182798012e-11, 3.377871924107149e-1..."


### Scikit learn vs PCA a partir de SVD aproximada mediante Jacobi-one sided

In [110]:
df_seccion_8_1_jac = comparaModelos.compara_resultados_abs_sk_jac(seccion_8_1)
df_seccion_8_1_jac.insert(0, "Seccion", np.repeat("seccion_8_1",4), True) 
df_seccion_8_1_jac

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_8_1,varianza explicada,False,9.148121e-02,"[0.01339421732388395, 0.02878380569739806, 0.0..."
1,seccion_8_1,valores singulares,False,4.683748e-02,"[0.006674831971013333, 0.014289803605153948, 0..."
2,seccion_8_1,coeficientes,False,2.732209e+02,"[[0.09084791344846525, 1.981559342620171, 0.79..."
3,seccion_8_1,componentes principales,False,6.790352e+06,"[[2335.163566647474, 29.851136890637655, 648.8..."


### Scikit learn vs PCA a partir del algoritmo QR

In [111]:
df_seccion_8_1_qr = comparaModelos.compara_resultados_abs_sk_qr(seccion_8_1)
df_seccion_8_1_qr.insert(0, "Seccion", np.repeat("seccion_8_1",4), True) 
df_seccion_8_1_qr

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_8_1,varianza explicada,True,0.000005,"[1.9432059455788525e-14, 4.538421461894037e-06..."
1,seccion_8_1,eigenvalores,True,0.000005,"[3.096087807917376e-15, 4.538421445469778e-06,..."
2,seccion_8_1,coeficientes,False,0.268490,"[[5.65430859569803e-13, 1.43675383091697e-13, ..."
3,seccion_8_1,componentes principales,False,28.790827,"[[2.881369681739466e-11, 1.0848434343838385, 0..."


In [175]:
np.linalg.cond(seccion_8_1)

1.3412687647074213

## 8.2. Maestros de Español (literatura, lectura y redacción, etcétera)

In [112]:
np.random.seed(150) ###Se define la semilla para la reproducibilidad de los resultados

seccion_8_2 = df.loc[:, ['p85', 'p86', 'p87', 'p88', 'p89']] ###Se seleccionan las preguntas que conforman esta sección

In [113]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()

seccion_8_2 = scaler.fit_transform(seccion_8_2)

### Scikit learn vs PCA a partir de SVD de numpy

In [114]:
df_seccion_8_2 = comparaModelos.compara_resultados_abs_sk_np(seccion_8_2)
df_seccion_8_2.insert(0, "Seccion", np.repeat("seccion_8_2",4), True) 
df_seccion_8_2

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_8_2,varianza explicada,True,2.083336e-16,"[0.0, 2.0833363867212563e-16]"
1,seccion_8_2,valores singulares,True,0.000000e+00,"[0.0, 0.0]"
2,seccion_8_2,coeficientes,True,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0...."
3,seccion_8_2,componentes principales,True,1.863889e-09,"[[1.863888878094738e-09, 2.960568365144985e-11..."


### Scikit learn vs PCA a partir de SVD aproximada mediante Jacobi-one sided

In [115]:
df_seccion_8_2_jac = comparaModelos.compara_resultados_abs_sk_jac(seccion_8_2)
df_seccion_8_2_jac.insert(0, "Seccion", np.repeat("seccion_8_2",4), True) 
df_seccion_8_2_jac

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_8_2,varianza explicada,False,0.000906,"[0.0009062626775334845, 2.1771322902853306e-05]"
1,seccion_8_2,valores singulares,False,0.000453,"[0.0004530287212364712, 1.0885602184156948e-05]"
2,seccion_8_2,coeficientes,False,5.682295,"[[0.026927109211328095, 0.0892985725717341, 1...."
3,seccion_8_2,componentes principales,False,776919.866459,"[[68.42147088965079, 674.7840741185718], [1446..."


### Scikit learn vs PCA a partir del algoritmo QR

In [116]:
df_seccion_8_2_qr = comparaModelos.compara_resultados_abs_sk_qr(seccion_8_2)
df_seccion_8_2_qr.insert(0, "Seccion", np.repeat("seccion_8_2",4), True) 
df_seccion_8_2_qr

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_8_2,varianza explicada,True,1.058335e-13,"[7.290159752027614e-14, 1.05833488445451e-13]"
1,seccion_8_2,eigenvalores,True,1.094644e-13,"[1.0946435733014267e-13, 6.933331127863261e-14]"
2,seccion_8_2,coeficientes,True,7.264399e-10,"[[1.022021937195438e-13, 4.760328772131606e-14..."
3,seccion_8_2,componentes principales,True,2.865539e-06,"[[1.863159971834557e-09, 2.2456987982850554e-1..."


## 8.3. Maestros de Matemáticas

In [117]:
seccion_8_3 = df.loc[:, ['p90', 'p91', 'p92', 'p93', 'p94', 'p95', 'p96', 'p97', 'p98', 'p99', 'p100', 'p101', 'p102', 'p103', 
                        'p104','p105','p106', 'p107', 'p108', 'p109', 'p110', 'p111', ]] ###Se seleccionan las preguntas que conforman esta sección

In [118]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()

seccion_8_3 = scaler.fit_transform(seccion_8_3)

### Scikit learn vs PCA a partir de SVD de numpy

In [119]:
df_seccion_8_3 = comparaModelos.compara_resultados_abs_sk_np(seccion_8_3)
df_seccion_8_3.insert(0, "Seccion", np.repeat("seccion_8_3",4), True) 
df_seccion_8_3

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_8_3,varianza explicada,True,1.489917e-16,"[1.4899167910446447e-16, 0.0, 0.0, 0.0, 0.0]"
1,seccion_8_3,valores singulares,True,0.000000e+00,"[0.0, 0.0, 0.0, 0.0, 0.0]"
2,seccion_8_3,coeficientes,True,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,seccion_8_3,componentes principales,True,1.936874e-08,"[[8.513937689139695e-12, 4.386529456891676e-11..."


### Scikit learn vs PCA a partir de SVD aproximada mediante Jacobi-one sided

In [120]:
df_seccion_8_3_jac = comparaModelos.compara_resultados_abs_sk_jac(seccion_8_3)
df_seccion_8_3_jac.insert(0, "Seccion", np.repeat("seccion_8_3",4), True) 
df_seccion_8_3_jac

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_8_3,varianza explicada,False,9.162654e-02,"[0.0036475491958196334, 0.026424498728868217, ..."
1,seccion_8_3,valores singulares,False,4.480933e-02,"[0.001822114547167448, 0.013126102086410973, 0..."
2,seccion_8_3,coeficientes,False,4.461264e+02,"[[0.08601135958769845, 0.46657735006960854, 0...."
3,seccion_8_3,componentes principales,False,1.333440e+09,"[[2997.078335858114, 370.7015529291581, 1355.9..."


### Scikit learn vs PCA a partir del algoritmo QR

In [121]:
df_seccion_8_3_qr = comparaModelos.compara_resultados_abs_sk_qr(seccion_8_3)
df_seccion_8_3_qr.insert(0, "Seccion", np.repeat("seccion_8_3",4), True) 
df_seccion_8_3_qr

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_8_3,varianza explicada,True,1.423646e-10,"[1.1919334328357301e-14, 1.7198058864426375e-1..."
1,seccion_8_3,eigenvalores,True,1.423720e-10,"[4.767725227039295e-15, 9.885083650522588e-15,..."
2,seccion_8_3,coeficientes,False,1.235319e-03,"[[7.397299498795681e-15, 4.2259671969557665e-1..."
3,seccion_8_3,componentes principales,False,8.141833e+01,"[[8.515406347980643e-12, 4.379788551719323e-11..."


In [176]:
np.linalg.cond(seccion_8_3)

5.43886042481056

## Sección 9. Entorno escolar

In [122]:
seccion_9 = df.loc[:, ['p112', 'p113', 'p114', 'p115', 'p116', 'p117','p118','p119', 'p120', 'p121', 'p122', 'p123',
                       'p124', 'p125','p126','p127', 'p128', 'p129', 'p130', 'p131']] ###Se seleccionan las preguntas que conforman esta sección

In [123]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()

seccion_9 = scaler.fit_transform(seccion_9)

### Scikit learn vs PCA a partir de SVD de numpy

In [124]:
df_seccion_9 = comparaModelos.compara_resultados_abs_sk_np(seccion_9)
df_seccion_9.insert(0, "Seccion", np.repeat("seccion_9",4), True) 
df_seccion_9

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_9,varianza explicada,True,1.974544e-16,"[1.97454408475595e-16, 0.0, 0.0, 0.0]"
1,seccion_9,valores singulares,True,0.000000e+00,"[0.0, 0.0, 0.0, 0.0]"
2,seccion_9,coeficientes,True,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,seccion_9,componentes principales,True,8.235779e-08,"[[6.815471630580412e-11, 3.868496048910068e-11..."


### Scikit learn vs PCA a partir de SVD aproximada mediante Jacobi-one sided

In [125]:
df_seccion_9_jac = comparaModelos.compara_resultados_abs_sk_jac(seccion_9)
df_seccion_9_jac.insert(0, "Seccion", np.repeat("seccion_9",4), True) 
df_seccion_9_jac

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_9,varianza explicada,False,9.266506e-02,"[0.0926650607573313, 0.05451657585589045, 0.08..."
1,seccion_9,valores singulares,False,4.530620e-02,"[0.04530620430442938, 0.02764028048046374, 0.0..."
2,seccion_9,coeficientes,False,5.716033e+01,"[[0.19766023612332118, 57.16033341125681, 0.20..."
3,seccion_9,componentes principales,False,1.141240e+09,"[[522.5822299432131, 1202.785365743722, 6182.9..."


### Scikit learn vs PCA a partir del algoritmo QR

In [126]:
df_seccion_9_qr = comparaModelos.compara_resultados_abs_sk_qr(seccion_9)
df_seccion_9_qr.insert(0, "Seccion", np.repeat("seccion_9",4), True) 
df_seccion_9_qr

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_9,varianza explicada,True,1.303488e-12,"[1.4019263001767446e-14, 2.356567841867348e-14..."
1,seccion_9,eigenvalores,True,1.313781e-12,"[2.4326339732729288e-14, 3.3771760918787334e-1..."
2,seccion_9,coeficientes,False,1.481188e-05,"[[1.4607206122587314e-14, 1.3211890052751638e-..."
3,seccion_9,componentes principales,False,2.616843e-01,"[[6.809677847444685e-11, 3.870338770668134e-11..."


In [177]:
np.linalg.cond(seccion_9)

4.856002780596158

# Sección 10. Entorno familiar

## 10.1. Entorno familiar

In [127]:
seccion_10_1 = df.loc[:, ['p132', 'p133', 'p134', 'p135', 'p136', 'p137', 'p138', 'p139', 'p140']
                     ] ###Se seleccionan las preguntas que conforman esta sección

In [128]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()

seccion_10_1 = scaler.fit_transform(seccion_10_1)

### Scikit learn vs PCA a partir de SVD de numpy

In [129]:
df_seccion_10_1 = comparaModelos.compara_resultados_abs_sk_np(seccion_10_1)
df_seccion_10_1.insert(0, "Seccion", np.repeat("seccion_10_1",4), True) 
df_seccion_10_1

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_10_1,varianza explicada,True,1.406767e-16,"[0.0, 0.0, 0.0, 1.4067672183145414e-16]"
1,seccion_10_1,valores singulares,True,0.000000e+00,"[0.0, 0.0, 0.0, 0.0]"
2,seccion_10_1,coeficientes,True,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]..."
3,seccion_10_1,componentes principales,True,8.505764e-09,"[[4.195470997628621e-11, 3.148937401122182e-11..."


### Scikit learn vs PCA a partir de SVD aproximada mediante Jacobi-one sided

In [130]:
df_seccion_10_1_jac = comparaModelos.compara_resultados_abs_sk_jac(seccion_10_1)
df_seccion_10_1_jac.insert(0, "Seccion", np.repeat("seccion_10_1",4), True) 
df_seccion_10_1_jac

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_10_1,varianza explicada,False,3.059724e-02,"[0.03059723734285997, 0.010807358290411975, 0...."
1,seccion_10_1,valores singulares,False,1.518335e-02,"[0.015183351588663728, 0.005389157635164663, 0..."
2,seccion_10_1,coeficientes,False,2.153746e+01,"[[0.46815517125038325, 0.21144397523613703, 0...."
3,seccion_10_1,componentes principales,False,1.442623e+08,"[[1390.2547512857423, 9489.941773978617, 114.9..."


### Scikit learn vs PCA a partir del algoritmo QR

In [131]:
df_seccion_10_1_qr = comparaModelos.compara_resultados_abs_sk_qr(seccion_10_1)
df_seccion_10_1_qr.insert(0, "Seccion", np.repeat("seccion_10_1",4), True) 
df_seccion_10_1_qr

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_10_1,varianza explicada,True,1.367378e-13,"[3.3980665033393925e-14, 4.7023532983460735e-1..."
1,seccion_10_1,eigenvalores,True,1.988227e-13,"[2.7981302072965103e-14, 1.4892323203429928e-1..."
2,seccion_10_1,coeficientes,True,3.650434e-10,"[[1.0482253215064124e-13, 4.779708528034512e-1..."
3,seccion_10_1,componentes principales,True,1.020546e-06,"[[4.205808927763966e-11, 3.134697043437474e-11..."


## 10.2. En tu casa, ¿cuentas con los siguientes bienes y servicios?

In [132]:
seccion_10_2 = df.loc[:, ['p141', 'p142', 'p143', 'p144', 'p145', 'p146', 'p147', 'p148', 'p149', 'p150', 'p151', 'p152',
                          'p153', 'p154']] ###Se seleccionan las preguntas que conforman esta sección

In [133]:
####Se escalan los datos.
scaler = preprocessing.StandardScaler()

seccion_10_2 = scaler.fit_transform(seccion_10_2)

### Scikit learn vs PCA a partir de SVD de numpy

In [134]:
df_seccion_10_2 = comparaModelos.compara_resultados_abs_sk_np(seccion_10_2)
df_seccion_10_2.insert(0, "Seccion", np.repeat("seccion_10_2",4), True) 
df_seccion_10_2

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_10_2,varianza explicada,True,1.899619e-16,"[0.0, 0.0, 0.0, 1.8996190611736057e-16, 0.0]"
1,seccion_10_2,valores singulares,True,0.000000e+00,"[0.0, 0.0, 0.0, 0.0, 0.0]"
2,seccion_10_2,coeficientes,True,0.000000e+00,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,seccion_10_2,componentes principales,True,1.206179e-08,"[[5.6704139146781604e-11, 2.9183583516211564e-..."


### Scikit learn vs PCA a partir de SVD aproximada mediante Jacobi-one sided

In [135]:
df_seccion_10_2_jac = comparaModelos.compara_resultados_abs_sk_jac(seccion_10_2)
df_seccion_10_2_jac.insert(0, "Seccion", np.repeat("seccion_10_2",4), True) 
df_seccion_10_2_jac

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_10_2,varianza explicada,False,4.577027e-01,"[0.4156117506365731, 0.4577027215644075, 0.006..."
1,seccion_10_2,valores singulares,False,2.635916e-01,"[0.18979483552273013, 0.2635916360906893, 0.00..."
2,seccion_10_2,coeficientes,False,1.504133e+02,"[[0.4340216516319654, 2.866630510233668, 0.077..."
3,seccion_10_2,componentes principales,False,1.860172e+09,"[[9771.063718072663, 3225.2562478124623, 5519...."


### Scikit learn vs PCA a partir del algoritmo QR

In [136]:
df_seccion_10_2_qr = comparaModelos.compara_resultados_abs_sk_qr(seccion_10_2)
df_seccion_10_2_qr.insert(0, "Seccion", np.repeat("seccion_10_2",4), True) 
df_seccion_10_2_qr

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_10_2,varianza explicada,True,8.199142e-11,"[8.49531232449764e-15, 7.378068021844639e-15, ..."
1,seccion_10_2,eigenvalores,True,8.198896e-11,"[5.990604692178757e-15, 4.864651556989874e-15,..."
2,seccion_10_2,coeficientes,False,1.065506e-02,"[[1.7776360554184003e-14, 3.525922210540173e-1..."
3,seccion_10_2,componentes principales,False,2.016112e+00,"[[5.667875817810058e-11, 2.4934963929074734e-1..."


In [178]:
np.linalg.cond(seccion_10_2)

3.338708924052618

# Resumen de resultados

### Scikit learn vs PCA a partir de SVD de numpy

Obtenemos las comparaciones faltantes con valores absolutos:

In [137]:
df_seccion_3 = comparaModelos.compara_resultados_abs_sk_np(seccion_3)
df_seccion_3.insert(0, "Seccion",np.repeat("seccion_3",4), True) 

In [138]:
sk_vs_np = [df_seccion_1,df_seccion_2,df_seccion_3, df_seccion_4, df_seccion_5_1,
 df_seccion_5_2, df_seccion_6, df_seccion_7_1, df_seccion_7_2,
 df_seccion_8_1, df_seccion_8_2, df_seccion_8_3,
 df_seccion_9, df_seccion_10_1, df_seccion_10_2]
df_sk_vs_np = pd.concat(sk_vs_np)

#### Valores singulares

Veamos los resultados de la comparación para los valores singulares:

In [139]:
# filter valores singulares 
df_sk_vs_np_val_sing = df_sk_vs_np[df_sk_vs_np['elemento']=='valores singulares']
df_sk_vs_np_val_sing

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
1,seccion_1,valores singulares,True,0.0,"[0.0, 0.0, 0.0]"
1,seccion_2,valores singulares,True,0.0,"[0.0, 0.0, 0.0]"
1,seccion_3,valores singulares,True,0.0,"[0.0, 0.0]"
1,seccion_4,valores singulares,True,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,seccion_5_1,valores singulares,True,0.0,[0.0]
1,seccion_5_2,valores singulares,True,0.0,"[0.0, 0.0]"
1,seccion_6,valores singulares,True,0.0,"[0.0, 0.0]"
1,seccion_7_1,valores singulares,True,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,seccion_7_2,valores singulares,True,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]"
1,seccion_8_1,valores singulares,True,0.0,"[0.0, 0.0, 0.0, 0.0]"


In [140]:
df_sk_vs_np_val_sing.shape

(15, 5)

#### Varianza explicada

Veamos los resultados de la comparación para la varianza explicada:

In [141]:
# filter valores singulares 
df_sk_vs_np_var_expl = df_sk_vs_np[df_sk_vs_np['elemento']=='varianza explicada']
df_sk_vs_np_var_expl

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_1,varianza explicada,True,1.845437e-16,"[1.8454368235801489e-16, 1.5513265898145355e-1..."
0,seccion_2,varianza explicada,True,0.000000e+00,"[0.0, 0.0, 0.0]"
0,seccion_3,varianza explicada,True,1.270989e-16,"[0.0, 1.2709886748169263e-16]"
0,seccion_4,varianza explicada,True,3.810244e-16,"[1.8163619603877741e-16, 2.6276477052749485e-1..."
0,seccion_5_1,varianza explicada,True,2.971219e-16,[2.9712188186743343e-16]
0,seccion_5_2,varianza explicada,True,1.873993e-16,"[1.873992524757673e-16, 0.0]"
0,seccion_6,varianza explicada,True,1.311963e-16,"[0.0, 1.3119626812763805e-16]"
0,seccion_7_1,varianza explicada,True,2.201664e-16,"[0.0, 1.1251842512572906e-16, 1.42763830014617..."
0,seccion_7_2,varianza explicada,True,2.178309e-16,"[2.178308965095565e-16, 1.2571561853136868e-16..."
0,seccion_8_1,varianza explicada,True,3.055070e-16,"[2.444284208275238e-16, 3.055070401062262e-16,..."


In [142]:
df_sk_vs_np_var_expl.shape

(15, 5)

En este caso, observamos que salvo por la sección 6, en ningún otro resultado obtuvimos la igualdad, y que los errores relativos son bastante grandes, por lo que no estamos obteniendo una muy buena aproximación.

#### Coeficientes

Veamos los resultados de la comparación para los coeficientes:

In [143]:
# filter valores singulares 
df_sk_vs_np_coef = df_sk_vs_np[df_sk_vs_np['elemento']=='coeficientes']
df_sk_vs_np_coef

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
2,seccion_1,coeficientes,True,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,seccion_2,coeficientes,True,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,seccion_3,coeficientes,True,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0...."
2,seccion_4,coeficientes,True,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,seccion_5_1,coeficientes,True,0.0,"[[0.0, 0.0, 0.0, 0.0]]"
2,seccion_5_2,coeficientes,True,0.0,"[[0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0]]"
2,seccion_6,coeficientes,True,0.0,"[[0.0, 0.0], [0.0, 0.0]]"
2,seccion_7_1,coeficientes,True,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,seccion_7_2,coeficientes,True,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,seccion_8_1,coeficientes,True,0.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0...."


In [144]:
df_sk_vs_np_coef.shape

(15, 5)

En este caso, observamos que salvo por la sección 6, en ningún otro resultado obtuvimos la igualdad, y que los errores relativos son bastante grandes, por lo que no estamos obteniendo una muy buena aproximación.

#### Componentes principales

Veamos los resultados de la comparación para los componentes obtenidos:

In [145]:
# filter valores singulares 
df_sk_vs_np_comp = df_sk_vs_np[df_sk_vs_np['elemento']=='componentes principales']
df_sk_vs_np_comp

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
3,seccion_1,componentes principales,True,1.328783e-09,"[[2.6344109242589875e-11, 2.6483430011396585e-..."
3,seccion_2,componentes principales,True,1.725904e-07,"[[6.02594840918337e-11, 2.0427044636858444e-09..."
3,seccion_3,componentes principales,True,1.254858e-09,"[[7.996555014011452e-12, 1.1263374054990842e-1..."
3,seccion_4,componentes principales,True,7.415905e-08,"[[7.267035848063278e-11, 5.408932705117198e-12..."
3,seccion_5_1,componentes principales,True,1.369991e-10,"[[1.0864534238555451e-10], [1.3710478122857844..."
3,seccion_5_2,componentes principales,True,1.499040e-09,"[[6.788500778143712e-11, 5.432152614613707e-12..."
3,seccion_6,componentes principales,True,1.080050e-12,"[[5.192048991001361e-13, 1.0800503623567881e-1..."
3,seccion_7_1,componentes principales,True,1.700735e-09,"[[8.415655220333915e-11, 6.328821898554283e-11..."
3,seccion_7_2,componentes principales,True,7.323949e-10,"[[9.761226003432158e-14, 1.0101021943151365e-1..."
3,seccion_8_1,componentes principales,True,7.033656e-11,"[[2.8732286182798012e-11, 3.377871924107149e-1..."


In [146]:
df_sk_vs_np_comp.shape

(15, 5)

Podemos verificar así que en todos los rubros los resultados fueron iguales para ambas funciones.

In [147]:
# Guardamos los resultados para mostrarlos en el reporte
compara_sk_np = [df_sk_vs_np_val_sing,df_sk_vs_np_var_expl,df_sk_vs_np_coef,df_sk_vs_np_comp]
df_compara_sk_np = pd.concat(compara_sk_np)
df_compara_sk_np.to_csv('../../results/data_results/comparativa_sk_np.csv', index=False)

### Scikit learn vs PCA a partir de SVD aproximada mediante Jacobi-one sided

Obtenemos las comparaciones faltantes con valores absolutos y juntamos los resultados en un data frame para facilitar su análisis:

In [148]:
df_seccion_3_jac = comparaModelos.compara_resultados_abs_sk_jac(seccion_3)
df_seccion_3_jac.insert(0, "Seccion", np.repeat("seccion_3",4), True) 

In [149]:
sk_vs_jac = [ df_seccion_1_jac,df_seccion_2_jac,df_seccion_3_jac, df_seccion_4_jac,
 df_seccion_5_1_jac, df_seccion_5_2_jac, df_seccion_6_jac, df_seccion_7_1_jac, 
 df_seccion_7_2_jac, df_seccion_8_1_jac, df_seccion_8_2_jac, df_seccion_8_3_jac,
 df_seccion_9_jac, df_seccion_10_1_jac, df_seccion_10_2_jac]
df_sk_vs_jac = pd.concat(sk_vs_jac)

#### Valores singulares

Veamos los resultados de la comparación para los valores singulares:

In [150]:
# filter valores singulares 
df_sk_vs_jac_val_sing = df_sk_vs_jac[df_sk_vs_jac['elemento']=='valores singulares']
df_sk_vs_jac_val_sing

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
1,seccion_1,valores singulares,False,2.489093e-01,"[0.24890934784595498, 0.014500133559060375, 0...."
1,seccion_2,valores singulares,False,1.082815e-01,"[0.06127626725063698, 0.10828148584329743, 0.0..."
1,seccion_3,valores singulares,False,5.315354e-02,"[0.01906552897109906, 0.05315353561094553]"
1,seccion_4,valores singulares,False,2.548450e-02,"[0.02035472727571438, 0.01639443449372074, 0.0..."
1,seccion_5_1,valores singulares,False,3.389791e-05,[3.389791155141556e-05]
1,seccion_5_2,valores singulares,False,2.105666e-03,"[0.0021056655253422533, 0.001643116161445561]"
1,seccion_6,valores singulares,True,4.947458e-15,"[4.947457968221809e-15, 3.466159070087135e-15]"
1,seccion_7_1,valores singulares,False,1.032035e-01,"[0.10320347771919386, 0.035728621517128425, 0...."
1,seccion_7_2,valores singulares,False,4.495770e-02,"[0.03635423469409431, 0.01669313253810829, 0.0..."
1,seccion_8_1,valores singulares,False,4.683748e-02,"[0.006674831971013333, 0.014289803605153948, 0..."


In [151]:
df_sk_vs_jac_val_sing.shape

(15, 5)

En este caso, observamos que salvo por la sección 6, donde la matriz tenía número de condición cercano a uno, en ningún otro resultado obtuvimos la igualdad, y que la mayoría de los errores relativos son alrededor del orden de $10^{-2}$.

#### Varianza explicada

Veamos los resultados de la comparación para la varianza explicada:

In [152]:
# filter valores singulares 
df_sk_vs_jac_var_expl = df_sk_vs_jac[df_sk_vs_jac['elemento']=='varianza explicada']
df_sk_vs_jac_var_expl

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_1,varianza explicada,False,5.597746e-01,"[0.5597745591370169, 0.028790013244885136, 0.0..."
0,seccion_2,varianza explicada,False,2.048381e-01,"[0.126307315429458, 0.20483809151015386, 0.114..."
0,seccion_3,varianza explicada,False,1.091324e-01,"[0.038494552337125544, 0.10913236956981359]"
0,seccion_4,varianza explicada,False,5.161847e-02,"[0.04112376947389873, 0.03252009150507147, 0.0..."
0,seccion_5_1,varianza explicada,False,6.779697e-05,[6.779697210380624e-05]
0,seccion_5_2,varianza explicada,False,4.215765e-03,"[0.00421576487797782, 0.003283532492182103]"
0,seccion_6,varianza explicada,True,1.705551e-15,"[1.1547088008302923e-15, 1.7055514856592915e-15]"
0,seccion_7_1,varianza explicada,False,2.170579e-01,"[0.21705791325173573, 0.07273377742978161, 0.1..."
0,seccion_7_2,varianza explicada,False,9.193659e-02,"[0.07403009976843317, 0.03310760440223537, 0.0..."
0,seccion_8_1,varianza explicada,False,9.148121e-02,"[0.01339421732388395, 0.02878380569739806, 0.0..."


In [153]:
df_sk_vs_jac_var_expl.shape

(15, 5)

De nuevo, salvo por la sección 6, en ningún otro resultado obtuvimos la igualdad, y la mayoría de los errores relativos son alrededor del orden de $10^{-2}$.

#### Coeficientes

Veamos los resultados de la comparación para los coeficientes:

In [154]:
# filter valores singulares 
df_sk_vs_jac_coef = df_sk_vs_jac[df_sk_vs_jac['elemento']=='coeficientes']
df_sk_vs_jac_coef

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
2,seccion_1,coeficientes,False,7.746779e+02,"[[0.8899611016386196, 0.7850258673183417, 0.79..."
2,seccion_2,coeficientes,False,3.730775e+01,"[[0.15104388965092766, 0.9311740542814365, 37...."
2,seccion_3,coeficientes,False,7.574608e+00,"[[0.4833494555008358, 0.019174802487839707, 0...."
2,seccion_4,coeficientes,False,6.948794e+02,"[[0.018741068235355388, 0.035586748186205185, ..."
2,seccion_5_1,coeficientes,False,6.011951e+00,"[[0.013487521683155597, 0.07487148385623248, 6..."
2,seccion_5_2,coeficientes,False,2.427195e+00,"[[0.12214631300002518, 0.2137900143185933, 0.1..."
2,seccion_6,coeficientes,True,8.635509e-15,"[[8.635508522760138e-15, 8.478499276892998e-15..."
2,seccion_7_1,coeficientes,False,6.989034e+01,"[[0.05572941838065679, 0.23890575665922156, 0...."
2,seccion_7_2,coeficientes,False,2.987641e+01,"[[0.6674372806598604, 0.4995088801926291, 1.07..."
2,seccion_8_1,coeficientes,False,2.732209e+02,"[[0.09084791344846525, 1.981559342620171, 0.79..."


In [155]:
df_sk_vs_jac_coef.shape

(15, 5)

En este caso, observamos que salvo por la sección 6, en ningún otro resultado obtuvimos la igualdad, y que los errores relativos son bastante grandes (hasta $10^2$), por lo que no estamos obteniendo una muy buena aproximación.

#### Componentes principales

Veamos los resultados de la comparación para los componentes obtenidos:

In [156]:
# filter valores singulares 
df_sk_vs_jac_comp = df_sk_vs_jac[df_sk_vs_jac['elemento']=='componentes principales']
df_sk_vs_jac_comp

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
3,seccion_1,componentes principales,False,6.549034e+06,"[[739.0213528847813, 30.11391147729844, 1154.1..."
3,seccion_2,componentes principales,False,2.998254e+06,"[[3380.405114797662, 55.090909932104154, 0.020..."
3,seccion_3,componentes principales,False,2.218037e+08,"[[1144.2424262714808, 3440.958961197482], [258..."
3,seccion_4,componentes principales,False,2.062674e+10,"[[1067.708263034393, 4624.663310507209, 1086.4..."
3,seccion_5_1,componentes principales,False,1.061854e+05,"[[2176.547143604811], [1185.3528509361165], [1..."
3,seccion_5_2,componentes principales,False,3.374886e+05,"[[3516.789333221952, 757.0460589487742], [3877..."
3,seccion_6,componentes principales,False,4.853796e+04,"[[761.0656750587331, 944.3492894702955], [86.4..."
3,seccion_7_1,componentes principales,False,6.646681e+06,"[[5278.56772669397, 2499.8443701227034, 29652...."
3,seccion_7_2,componentes principales,False,4.885065e+07,"[[1092.6131895026354, 1179.0850653572131, 1280..."
3,seccion_8_1,componentes principales,False,6.790352e+06,"[[2335.163566647474, 29.851136890637655, 648.8..."


En el caso de los componentes principales, en ninguna sección obtuvimos la igualdad respecto al resultado de `sk learn`, y además, los errores relativos se ven magnificados hasta un orden de $10^{10}$, por lo que no estamos obteniendo una muy buena aproximación.

In [157]:
# Guardamos los resultados para mostrarlos en el reporte
compara_sk_jacobi = [df_sk_vs_jac_val_sing,df_sk_vs_jac_var_expl,df_sk_vs_jac_coef,df_sk_vs_jac_comp]
df_compara_sk_jacobi = pd.concat(compara_sk_jacobi)
# df_compara_sk_jacobi.to_csv('../../results/data_results/comparativa_sk_jacobi.csv', index=False)

### Scikit learn vs PCA a partir del algoritmo QR

Obtenemos las comparaciones faltantes con valores absolutos y juntamos los resultados en un data frame para facilitar su análisis:

In [181]:
sk_vs_qr = [ df_seccion_1_qr,df_seccion_2_qr,df_seccion_3_qr, df_seccion_4_qr,
 df_seccion_5_1_qr, df_seccion_5_2_qr, df_seccion_6_qr, df_seccion_7_1_qr, 
 df_seccion_7_2_qr, df_seccion_8_1_qr, df_seccion_8_2_qr, df_seccion_8_3_qr,
 df_seccion_9_qr, df_seccion_10_1_qr, df_seccion_10_2_qr]
df_sk_vs_qr = pd.concat(sk_vs_qr)

#### Valores singulares

Veamos los resultados de la comparación para los valores singulares:

In [189]:
# filter valores singulares 
df_sk_vs_qr_eigenval = df_sk_vs_qr[df_sk_vs_qr['elemento']=='eigenvalores']
df_sk_vs_qr_eigenval

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
1,seccion_1,eigenvalores,True,1.402497e-13,"[6.469083362511687e-14, 2.414423985846565e-14,..."
1,seccion_2,eigenvalores,True,6.182706e-06,"[1.3636577603894522e-13, 6.016401196106755e-06..."
1,seccion_3,eigenvalores,True,5.348311e-14,"[1.9301538251445997e-14, 5.3483108037220185e-14]"
1,seccion_4,eigenvalores,False,3.287058e-03,"[6.919461792403764e-15, 5.00503432054534e-15, ..."
1,seccion_5_1,eigenvalores,True,1.384585e-13,[1.3845854997846134e-13]
1,seccion_5_2,eigenvalores,True,6.158501e-14,"[2.0426482084662247e-14, 6.158501343219229e-14]"
1,seccion_6,eigenvalores,True,9.815007e-15,"[9.815007299798092e-15, 7.08458584195003e-15]"
1,seccion_7_1,eigenvalores,False,1.525962e-03,"[6.870259717438163e-14, 2.5879191617594965e-15..."
1,seccion_7_2,eigenvalores,False,NaN,"[nan, nan, nan, nan, nan, nan]"
1,seccion_8_1,eigenvalores,True,4.667493e-06,"[3.096087807917376e-15, 4.538421445469778e-06,..."


In [190]:
df_sk_vs_qr_eigenval.shape

(15, 5)

En este caso, observamos que salvo por la sección 6, donde la matriz tenía número de condición cercano a uno, en ningún otro resultado obtuvimos la igualdad, y que la mayoría de los errores relativos son alrededor del orden de $10^{-2}$.

#### Varianza explicada

Veamos los resultados de la comparación para la varianza explicada:

In [184]:
# filter valores singulares 
df_sk_vs_qr_var_expl = df_sk_vs_qr[df_sk_vs_qr['elemento']=='varianza explicada']
df_sk_vs_qr_var_expl

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
0,seccion_1,varianza explicada,True,1.357242e-13,"[6.016124044870922e-14, 2.869954191156973e-14,..."
0,seccion_2,varianza explicada,True,6.182706e-06,"[1.426533869698987e-13, 6.0164012023457805e-06..."
0,seccion_3,varianza explicada,True,3.164762e-14,"[2.5690750673682666e-15, 3.164761800294246e-14]"
0,seccion_4,varianza explicada,False,3.287058e-03,"[9.081809801938787e-15, 2.7590300905387026e-15..."
0,seccion_5_1,varianza explicada,True,7.026933e-14,[7.026932506165296e-14]
0,seccion_5_2,varianza explicada,True,7.398868e-14,"[8.058167856458061e-15, 7.398868441239215e-14]"
0,seccion_6,varianza explicada,True,1.574355e-15,"[1.1547088008302923e-15, 1.574355217531654e-15]"
0,seccion_7_1,varianza explicada,False,1.525962e-03,"[6.593987576421898e-14, 5.513402831160754e-15,..."
0,seccion_7_2,varianza explicada,False,NaN,"[nan, nan, nan, nan, nan, nan]"
0,seccion_8_1,varianza explicada,True,4.667493e-06,"[1.9432059455788525e-14, 4.538421461894037e-06..."


In [185]:
df_sk_vs_qr_var_expl.shape

(15, 5)

De nuevo, salvo por la sección 6, en ningún otro resultado obtuvimos la igualdad, y la mayoría de los errores relativos son alrededor del orden de $10^{-2}$.

#### Coeficientes

Veamos los resultados de la comparación para los coeficientes:

In [186]:
# filter valores singulares 
df_sk_vs_qr_coef = df_sk_vs_qr[df_sk_vs_qr['elemento']=='coeficientes']
df_sk_vs_qr_coef

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
2,seccion_1,coeficientes,True,8.808192e-06,"[[1.716471218079173e-14, 1.8088529315322355e-1..."
2,seccion_2,coeficientes,False,2.135141e-01,"[[9.63301597215641e-14, 2.389524864392304e-13,..."
2,seccion_3,coeficientes,True,3.237183e-08,"[[1.5891228936468213e-13, 1.5184937284479278e-..."
2,seccion_4,coeficientes,False,5.538139e+01,"[[5.809725747878342e-15, 6.155074707948904e-14..."
2,seccion_5_1,coeficientes,True,2.924084e-13,"[[2.9240838097875275e-13, 3.002608367843295e-1..."
2,seccion_5_2,coeficientes,True,2.069570e-12,"[[5.488142467074827e-14, 9.080326838272184e-14..."
2,seccion_6,coeficientes,True,1.139759e-08,"[[1.1397585057405627e-08, 1.1397583747124481e-..."
2,seccion_7_1,coeficientes,False,9.211444e+01,"[[4.938881230546565e-10, 1.1860579865672855e-1..."
2,seccion_7_2,coeficientes,False,NaN,"[[nan, nan, nan, nan, nan, nan, nan, nan, nan,..."
2,seccion_8_1,coeficientes,False,2.684905e-01,"[[5.65430859569803e-13, 1.43675383091697e-13, ..."


In [155]:
df_sk_vs_qr_coef.shape

(15, 5)

En este caso, observamos que salvo por la sección 6, en ningún otro resultado obtuvimos la igualdad, y que los errores relativos son bastante grandes (hasta $10^2$), por lo que no estamos obteniendo una muy buena aproximación.

#### Componentes principales

Veamos los resultados de la comparación para los componentes obtenidos:

In [187]:
# filter valores singulares 
df_sk_vs_qr_comp = df_sk_vs_qr[df_sk_vs_qr['elemento']=='componentes principales']
df_sk_vs_qr_comp

,Seccion,elemento,Igualdad (en valor absoluto),Max error relativo (con valor absoluto),Error relativo (con valor absoluto)
3,seccion_1,componentes principales,False,1.790270e-04,"[[2.6356809626567568e-11, 6.134861477385035e-0..."
3,seccion_2,componentes principales,False,2.513791e+00,"[[6.024223679040742e-11, 0.0001648771947033299..."
3,seccion_3,componentes principales,True,1.026288e-05,"[[7.979283139599526e-12, 2.1940877302197085e-0..."
3,seccion_4,componentes principales,False,3.624384e+05,"[[7.27210338561804e-11, 5.398044704011588e-12,..."
3,seccion_5_1,componentes principales,True,1.086433e-10,"[[1.086432785018871e-10], [1.3621290702737174e..."
3,seccion_5_2,componentes principales,True,9.057784e-10,"[[6.780675230636451e-11, 5.478113032138733e-12..."
3,seccion_6,componentes principales,True,1.016404e-06,"[[4.807996512030944e-09, 2.7022518785506908e-0..."
3,seccion_7_1,componentes principales,False,5.979786e+02,"[[2.761215552029329e-10, 1.3308003747017288e-1..."
3,seccion_7_2,componentes principales,False,NaN,"[[nan, nan, nan, nan, nan, nan], [nan, nan, na..."
3,seccion_8_1,componentes principales,False,2.879083e+01,"[[2.881369681739466e-11, 1.0848434343838385, 0..."


In [188]:
df_sk_vs_qr_comp.shape

(15, 5)

En el caso de los componentes principales, en ninguna sección obtuvimos la igualdad respecto al resultado de `sk learn`, y además, los errores relativos se ven magnificados hasta un orden de $10^{10}$, por lo que no estamos obteniendo una muy buena aproximación.

In [191]:
# Guardamos los resultados para mostrarlos en el reporte
compara_sk_qr = [df_sk_vs_qr_eigenval,df_sk_vs_qr_var_expl,df_sk_vs_qr_coef,df_sk_vs_qr_comp]
df_compara_sk_qr = pd.concat(compara_sk_qr)
df_compara_sk_qr.to_csv('../../results/data_results/comparativa_sk_qr.csv', index=False)